### xml to txt file transform

In [1]:
import numpy as np
import lxml
import os

from lxml import etree

In [2]:
dataPath = 'C:/Users/SBA/git_ws/mini-highway/xmltotxt/highway_dataset'
dataPath_t = '/Training/[라벨]1.수도권영동선/'
dataPath_v = '/Validation/[라벨]1.수도권영동선/'

In [3]:
CLASSES = ["car", "bus", "truck"]

def to_yolov8(y):
  """
  # change to yolo v8 format
  # [x_top_left, y_top_left, x_bottom_right, y_bottom_right] to
  # [x_center, y_center, width, height]
  """
  width = y[2] - y[0]
  height = y[3] - y[1]

  if width < 0 or height < 0:
      print("ERROR: negative width or height ", width, height, y)
      raise AssertionError("Negative width or height")
  return (y[0] + (width/2)), (y[1] + (height/2)), width, height

def load_xml_annotations(f):
  tree = etree.parse(f)
  anns = []
  for dim in tree.xpath("image"):
    image_filename = dim.attrib["name"]
    width = int(dim.attrib["width"])
    height = int(dim.attrib["height"])
    # print(image_filename)
    # print(len(dim.xpath("box")))
    boxes = []
    for box in dim.xpath("box"):
      label = CLASSES.index(box.attrib["label"])
      xtl, ytl = box.attrib["xtl"], box.attrib["ytl"]
      xbr, ybr = box.attrib["xbr"], box.attrib["ybr"]

      xc, yc, w, h = to_yolov8([float(xtl), float(ytl), float(xbr), float(ybr)])
      boxes.append([label, round(xc/width, 5), round(yc/height, 5), round(w/width, 5), round(h/height, 5)])

    anns.append([image_filename, width, height, boxes])

  return np.array(anns, dtype=object)

In [4]:
def write_yolov8_txt(folder, annotation):
  out_filename = str(folder + annotation[0][:-3])
  out_filename = os.path.splitext(out_filename)[0]
  out_filename = out_filename+'.txt'

  f = open(out_filename,"w")
  
  for box in annotation[3]:
    f.write("{} {} {} {} {}\n".format(box[0], box[1], box[2], box[3], box[4]))

In [24]:
dataPath+dataPath_t
dataPath

'C:/Users/SBA/git_ws/mini-highway/xmltotxt/highway_dataset'

### 변환

In [36]:
label_file = os.path.join(dataPath+dataPath_t,'Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD.xml')
#print(label_file)

anns = load_xml_annotations(label_file)
#print(anns)

#for ann in anns:
#    write_yolov8_txt(dataPath + '/Training/transform_label/', ann)
    

[['Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_001.png' 1920
  1080
  list([[2, 0.70172, 0.54471, 0.02857, 0.04529], [2, 0.55315, 0.86765, 0.05567, 0.10332], [2, 0.57988, 0.60055, 0.04291, 0.09265], [2, 0.64396, 0.64562, 0.0337, 0.05186], [0, 0.62288, 0.57988, 0.02068, 0.03288], [0, 0.65474, 0.7765, 0.04248, 0.08641], [0, 0.54397, 0.68912, 0.03223, 0.0533], [2, 0.71325, 0.53245, 0.03927, 0.0419], [0, 0.81297, 0.89938, 0.08124, 0.11747]])]
 ['Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_002.png' 1920
  1080
  list([[2, 0.72392, 0.54317, 0.0352, 0.04229], [0, 0.54416, 0.76741, 0.04219, 0.06923], [0, 0.65091, 0.66137, 0.03287, 0.04262], [0, 0.67683, 0.63179, 0.03062, 0.04856], [0, 0.82037, 0.90708, 0.08596, 0.12991], [2, 0.63782, 0.50617, 0.03187, 0.04524], [0, 0.84431, 0.70379, 0.05141, 0.04978], [2, 0.72237, 0.57932, 0.02878, 0.03425]])]
 ['Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_003.png' 1920
  1080
  list([[0, 0.25626, 0.19439, 0.03299, 0.0446

In [63]:
xml_files = os.listdir(dataPath+dataPath_t)
xml_files_ch1 = [file for file in xml_files if file.startswith('Suwon_CH01')]
xml_files_ch2 = [file for file in xml_files if file.startswith('Suwon_CH02')]
xml_files_ch3 = [file for file in xml_files if file.startswith('Suwon_CH03')]
xml_files_ch4 = [file for file in xml_files if file.startswith('Suwon_CH04')]
#xml_files_ch

anns = load_xml_annotations(xml_files_ch1)


['Suwon_CH01_20200720_1700_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_1800_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_1900_MON_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_1930_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_2000_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_2030_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1530_TUE_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1600_TUE_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1630_TUE_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1730_TUE_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1800_TUE_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1900_TUE_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1930_TUE_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_203

In [71]:
xml_files = os.listdir(os.path.join(dataPath+dataPath_t))
xml_files

['Suwon_CH01_20200720_1700_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_1800_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_1900_MON_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_1930_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_2000_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200720_2030_MON_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1530_TUE_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1600_TUE_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1630_TUE_9m_NH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1730_TUE_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1800_TUE_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1900_TUE_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_1930_TUE_9m_RH_highway_TW5_sunny_FHD.xml',
 'Suwon_CH01_20200721_203

### 정리
- CH01 - CH04까지의 xml 파일만 사용
- CH01 - CH04는 총 168개의 폴더로 구성
    - CH01은 48개
    - CH02는 42개
    - CH03은 42개
    - CH04는 41개
- 받은 소스코드는 1개의 xml 파일을 여러개로 나눈 txt파일로 구성되는 코드 
- 1개의 파일은 1개의 폴더로 만들고 내부에는 여러개의 txt파일로 구성되어야 함

In [5]:
def convert_xml_to_txt(dataPath, dataPath_t, outPath, ch):
    xml_files = os.listdir(os.path.join(dataPath, dataPath_t))
    xml_files = [file for file in xml_files if file.startswith(ch)]
    for xml_file in xml_files:
        ann = load_xml_annotations(os.path.join(dataPath, dataPath_t, xml_file))
        out_folder = os.path.join(outPath, ch, xml_file[:-4])
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)

        write_yolov8_txt(outPath+ch, ann)